<a href="https://colab.research.google.com/github/AndrewZhang76/gnn_with_spmm/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10-414/714: Deep Learning Systems - Final Project


## **Sparse Matrix Multiplication on Graph Neural Network**

**By Andrew Zhang, Jinkai Qiu & Yimei Wu**


---

In this project, we are going to implement **sparse matrix** class supported in Needle, **forward and backward pass of sparse matrix multiplication**, and its **application on Graph Neural Network(GNN)**.

In this notebook, we are going to show how to **define sparse matrix**, perform **sparse matrix multiplication** and compare it with normal dense matrix multiplication. In addition, we will also show how it can be used in **GNN training.**



## 1. Clone Required Repo and Install Required Packages

In [1]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/10714
!mkdir -p final_proj
%cd /content/drive/MyDrive/10714/final_proj
!git clone https://github.com/AndrewZhang76/gnn_with_spmm.git
%cd gnn_with_spmm/
!git pull
!pip3 install pybind11

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/10714
/content/drive/MyDrive/10714/final_proj
fatal: destination path 'gnn_with_spmm' already exists and is not an empty directory.
/content/drive/MyDrive/10714/final_proj/gnn_with_spmm
Already up to date.


## 2. Build

In [2]:
%cd /content/drive/MyDrive/10714/final_proj/gnn_with_spmm/
!make

/content/drive/MyDrive/10714/final_proj/gnn_with_spmm
-- Found pybind11: /usr/local/lib/python3.10/dist-packages/pybind11/include (found version "2.13.6")
CMake Warning (dev) at CMakeLists.txt:57 (find_package):
  Policy CMP0146 is not set: The FindCUDA module is removed.  Run "cmake
  --help-policy CMP0146" for policy details.  Use the cmake_policy command to
  set the policy and suppress this warning.

This warning is for project developers.  Use -Wno-dev to suppress it.

-- CUDA_FOUND: TRUE
-- Found cuda, building cuda backend
Thu Dec 12 02:20:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-U

In [3]:
%set_env PYTHONPATH ./python
%set_env NEEDLE_BACKEND nd

env: PYTHONPATH=./python
env: NEEDLE_BACKEND=nd


## 3. Sparse Matrix Definiation.
In this project, we defined a new way to represent a sparse matrix(a type of matrix that contains a significant number of zero elements compared to the total number of elements in the matrix.) - **COO (Coordinate) format**. \
The COO (Coordinate) format is a representation of a sparse matrix that stores only the nonzero elements along with their row and column indices. It is efficient in terms of memory usage for sparse matrices because it avoids storing zero values.\
### Key Components:


1.   Values (data): A list or array of the nonzero elements in the matrix.
2.   Row indices (row): A list or array specifying the row index for each nonzero element.
3. Column indices (col): A list or array specifying the column index for each nonzero element.

###Properties:
1. Flexible: Allows easy manipulation, such as matrix construction from nonzero entries.
2. Duplicates: COO format allows duplicate entries. To obtain the actual matrix, these duplicates need to be summed.
3. Conversion: Often converted to other formats like CSR (Compressed Sparse Row) or CSC (Compressed Sparse Column) for efficient matrix operations.






#### First, we are going to randomly generate a sparse matrix in normal NDArray format. It is a 10 * 10 matrix with 10 non-zero elements.

In [4]:
%cd /content/drive/MyDrive/10714/final_proj/gnn_with_spmm/python/needle
import numpy as np
from backend_ndarray.ndarray import *

np.random.seed(0)
device = cuda()
# Dimensions of the matrix
rows, cols = 10, 10
nonzero_elements = 10

# Initialize a sparse matrix with all zeros
matrix = np.zeros((rows, cols))

# Randomly generate indices for nonzero elements
row_indices = np.random.choice(rows, nonzero_elements, replace=True)
col_indices = np.random.choice(cols, nonzero_elements, replace=True)

# Generate random values for the nonzero elements
values = np.random.random(nonzero_elements)

# Populate the matrix
for r, c, v in zip(row_indices, col_indices, values):
    matrix[r, c] = v

orig_matrix = NDArray(matrix, device=device)
orig_matrix

/content/drive/MyDrive/10714/final_proj/gnn_with_spmm/python/needle


NDArray([[0.         0.         0.         0.         0.         0.
  0.07103606 0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.7991586  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.87001216 0.0202184  0.        ]
 [0.         0.46147937 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.9786183  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.83261985 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.   

#### Then, we are going to transform it to a sparse matrix.

In [5]:
sparse_matrix = orig_matrix.to_sparse()
sparse_matrix

SparseMatrix(nnz=8, shape=(10, 10),
  data=[0.07103606 0.7991586  0.87001216 0.0202184  0.46147937 0.9786183
 0.83261985 0.77815676],
  row_indices=[0 2 3 3 4 5 7 9],
  col_indices=[6 8 7 8 1 7 1 6])

As you can see, the sparse matrix contains three length 10 array, `data`, `row_indices` and `col_indices`. The `data` represents the values of the all the non-zero elements inside the matrix, while `row_indices` and `col_indices` represents the row and column index of each non-zero element's index.

We can also switch the sparse matrix back to dense matrix by calling `to_dense()` function.



In [6]:
dense_matrix = sparse_matrix.to_dense()
dense_matrix

NDArray([[0.         0.         0.         0.         0.         0.
  0.07103606 0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.7991586  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.87001216 0.0202184  0.        ]
 [0.         0.46147937 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.9786183  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.83261985 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.   

## 4. Sparse Matrix Multiplication
### COO Format Matrix Multiplication

Matrix multiplication in the **COO (Coordinate)** format involves multiplying two sparse matrices that are represented in the COO format. Since the COO format only stores nonzero elements along with their row and column indices, performing multiplication requires processing each nonzero element and its corresponding coordinates.

#### Step 1: Represent the Matrices in COO Format
Assume two matrices A and B, both stored in COO format. They are represented by the following components:

- **Matrix A**:
  - `A_data`: Nonzero values in matrix A.
  - `A_row`: Row indices of the nonzero values in A.
  - `A_col`: Column indices of the nonzero values in A.

- **Matrix B**:
  - `B_data`: Nonzero values in matrix B.
  - `B_row`: Row indices of the nonzero values in B.
  - `B_col`: Column indices of the nonzero values in B.

Let matrix A be of size m × n and matrix B be of size n × p. The resulting matrix C will be of size m × p.

#### Step 2: Initialize the Resultant Matrix
The resulting matrix C will also be sparse and initially contain only zeros. In COO format, the result will have:

- `C_data`: Nonzero values of the resulting matrix.
- `C_row`: Row indices of the nonzero values in C.
- `C_col`: Column indices of the nonzero values in C.

#### Step 3: Compute Nonzero Elements of the Result
To calculate C = A × B, follow these steps for each nonzero element of A:

1. **Find the corresponding element in matrix B**: For each nonzero element A_ij in A, find the corresponding column indices of B. The row index in B must match the column index in A to compute the dot product.

2. **Perform the multiplication**: For each pair of nonzero elements A_ij and B_jk, multiply them together and add to the corresponding entry in C:
   
   ```
   C_ik = C_ik + A_ij × B_jk
   ```

3. **Store the result**: If C_ik is nonzero after the above addition, store the result in the COO format:
   - Add the value of C_ik to `C_data`.
   - Add the row index i to `C_row`.
   - Add the column index k to `C_col`.

#### Step 4: Handle Sparse Properties
Since the result matrix C is also sparse, ensure that only nonzero values are stored. If the sum of C_ik is zero, it should not be stored in the COO format.
#### Define two sparse matrices.


In [7]:
m, n, p = 500, 500, 500
device = cuda()
nnz = 100
# Initialize a sparse matrix with all zeros
matrix1 = np.zeros((m, n))
matrix2 = np.zeros((n, p))

# Randomly generate indices for nonzero elements
row_indices_1 = np.random.choice(m, nnz, replace=True)
col_indices_1 = np.random.choice(n, nnz, replace=True)
row_indices_2 = np.random.choice(n, nnz, replace=True)
col_indices_2 = np.random.choice(p, nnz, replace=True)

# Generate random values for the nonzero elements
values_1 = np.random.random(nnz)
values_2 = np.random.random(nnz)

# Populate the matrix
for r, c, v in zip(row_indices_1, col_indices_2, values_1):
    matrix1[r, c] = v
for r, c, v in zip(row_indices_2, col_indices_2, values_2):
    matrix2[r, c] = v

dense_matrix1 = NDArray(matrix1, device=device)
dense_matrix2 = NDArray(matrix2, device=device)
dense_matrix1, dense_matrix2

(NDArray([[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]], device=cuda()),
 NDArray([[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]], device=cuda()))

In [8]:
# Convert dense matrices to sparse matrices.
sparse_matrix1 = dense_matrix1.to_sparse()
sparse_matrix2 = dense_matrix2.to_sparse()
sparse_matrix1, sparse_matrix2

(SparseMatrix(nnz=100, shape=(500, 500),
   data=[0.3960597  0.37416998 0.76532525 0.6304479  0.58641016 0.10029394
  0.46357542 0.29214752 0.40724117 0.05537432 0.35561273 0.4783703
  0.13248764 0.96157014 0.6178767  0.23170163 0.07952208 0.77058077
  0.79920256 0.1314828  0.96193635 0.97749513 0.14484777 0.63947254
  0.42468548 0.29302028 0.18327984 0.49739137 0.8638556  0.22431703
  0.9473706  0.09784448 0.21550767 0.72559434 0.11753186 0.05342718
  0.82211775 0.20747007 0.7168597  0.24536721 0.01323686 0.14694664
  0.9065555  0.27762872 0.8621915  0.58678436 0.9037197  0.08960304
  0.9295293  0.14814086 0.02566272 0.87650526 0.08342244 0.9608347
  0.21331197 0.42053947 0.5654213  0.06395527 0.8605512  0.23223414
  0.66991657 0.3472335  0.08110139 0.511319   0.55219245 0.13690028
  0.7270443  0.4856276  0.6720478  0.26211816 0.33314514 0.13206811
  0.01642963 0.9493188  0.2703279  0.58447605 0.94043195 0.87428796
  0.7740473  0.7308558  0.9413777  0.68328136 0.5573688  0.48805627
  

#### Matrix Multiplication between dense matrices.

In [9]:
import time
start = time.time()
correct_result = dense_matrix1 @ dense_matrix2
end = time.time()
print(f"Time taken for dense-dense matrix multipliction: {(end - start)*1000} ms")
correct_result

Time taken for dense-dense matrix multipliction: 0.5300045013427734 ms


NDArray([[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], device=cuda())

#### Matrix Multiplication between sparse matrices.

In [10]:
start = time.time()
result = sparse_matrix1 @ sparse_matrix2
print(result)
# result = result.to_dense() # when flag = True return dense
end = time.time()
print(f"Time taken for sparse-sparse matrix multipliction: {(end - start)*1000} ms")
print(f"Result Correction: \n{np.allclose(correct_result.numpy(), result.numpy())}")

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Time taken for sparse-sparse matrix multipliction: 2.179384231567383 ms
Result Correction: 
True


In [11]:
start = time.time()
result = sparse_matrix1 @ dense_matrix2
result = result
end = time.time()
print(f"Time taken for sparse-dense matrix multipliction: {(end - start)*1000} ms")
print(f"Result Correction: \n{np.allclose(correct_result.numpy(), result.numpy())}")

Time taken for sparse-dense matrix multipliction: 0.6663799285888672 ms
Result Correction: 
True


In [12]:
start = time.time()
result = dense_matrix1 @ sparse_matrix2
result = result
end = time.time()
print(f"Time taken for dense-sparse matrix multipliction: {(end - start)*1000} ms")
print(f"Result Correction: \n{np.allclose(correct_result.numpy(), result.numpy())}")

Time taken for dense-sparse matrix multipliction: 0.32329559326171875 ms
Result Correction: 
True


## Benchmark Test
As we can see from the previous part, with a small matrix that is not very sparse(10/100), the benefits of using sparse matrix can not be shown. In the following cell, we run a test on 100 * 100 matrix with different number of non-zero elements, the acceleration result can be shown from it.

In [13]:
!python -m backend_ndarray.ndarray

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'backend_ndarray.ndarray' found in sys.modules after import of package 'backend_ndarray', but prior to execution of 'backend_ndarray.ndarray'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
Test Case: Size = ( 100 x 100 ), number of non-zero elements: 25
Time for dense @ dense: 0.07245540618896484 ms
Time for sparse @ dense: 0.05536079406738281 ms
Time for sparse @ sparse: 0.04115104675292969 ms
Test Case: Size = ( 100 x 100 ), number of non-zero elements: 100
Time for dense @ dense: 0.06649494171142578 ms
Time for sparse @ dense: 0.050902366638183594 ms
Time for sparse @ sparse: 0.03948211669921875 ms
Test Case: Size = ( 100 x 100 ), number of non-zero elements: 225
Time for dense @ dense: 0.06666183471679688 ms
Time for sparse @ dense: 0.06313323974609375 ms
Time for sparse @ sparse: 0.042366981506347656 ms


## Graph Neural Network(GNN)
A Graph Neural Network (GNN) is a type of neural network specifically designed to process and analyze data structured as graphs. In a graph, data is represented as nodes (vertices) connected by edges (links), which may have associated features or attributes.
### Common GNN Architectures:
1. Graph Convolutional Networks (GCN): Generalizes the concept of convolution to graphs.
2. Graph Attention Networks (GAT): Uses attention mechanisms to weigh neighbor contributions.
3. GraphSAGE: Focuses on efficient sampling and aggregating information from neighbors.

In this project, we are going to implement Graph Convolutional Network(GCN) and use sparse matrix multiplication mechnisms we implemented to accelerate its forward and backward passes.

### 1. Restart session and build file

In [1]:
%cd /content/drive/MyDrive/10714/final_proj/gnn_with_spmm
!make
%set_env PYTHONPATH ./python
%set_env NEEDLE_BACKEND nd

/content/drive/MyDrive/10714/final_proj/gnn_with_spmm
-- Found pybind11: /usr/local/lib/python3.10/dist-packages/pybind11/include (found version "2.13.6")
CMake Warning (dev) at CMakeLists.txt:57 (find_package):
  Policy CMP0146 is not set: The FindCUDA module is removed.  Run "cmake
  --help-policy CMP0146" for policy details.  Use the cmake_policy command to
  set the policy and suppress this warning.

This warning is for project developers.  Use -Wno-dev to suppress it.

-- CUDA_FOUND: TRUE
-- Found cuda, building cuda backend
Thu Dec 12 02:21:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-U

### 2. Processing Cora dataset

The Cora dataset is one of the most widely used benchmarks for evaluating graph-based machine learning models, especially Graph Neural Networks (GNNs). It is a citation network, where nodes represent scientific publications and edges represent citation relationships between them

**Dataset Overview**

*   Nodes: Publications (papers, books, etc.)
*   Edges: Citation relationships between publications
*   Node Features: Word vectors (1433 dimensions) representing the presence/absence of words from a dictionary
*   Labels: 7 classes representing publication types (e.g., Neural Networks, Reinforcement Learning, etc.)

**Key Statistics**

*   Number of graph: 1
*   Number of Nodes: 2708
*   Number of Edges: 5429 (undirected)
*   Number of Features: 1433
*   Number of Classes: 7

**Implementation**

The Cora dataset is processed using a class CoraDataset, implemented in cora_dataset.py under the ./python/needle/data directory. This implementation reads the raw data files (./python/data/cora/cora.content and ./python/data/cora/cora.cites), extracts the features, labels, and graph structure, and processes them into formats suitable for GNN task. The CoraDataset class is designed to load, process, and store the Cora dataset’s key components.

### 3. Implementing Graph Convolutional Neural Network

The `nn_gnn.py` module implements a two layer Graph Convolutional Network architecture, focusing on two key classes: `GraphConvolution` and `simpleGCN`. We enable graph-based neural network operations by extending traditional neural network layers to work with graph-structured data.

#### GraphConvolution Layer

The `GraphConvolution` layer serves as the fundamental building block of graph neural networks. We design this layer to efficiently transform node features using graph adjacency information. Key implementation details include:

**Learnable Parameters**
- Weight matrix initialized using Kaiming uniform initialization
- Optional bias term with flexible initialization

**Sparse Matrix Operations**
- Performs sparse matrix multiplication between graph adjacency matrix and node features
- Enables efficient processing of large, sparse graph structures

**Forward Propagation Mechanism**
1. Convert adjacency matrix to sparse representation
2. Perform sparse matrix multiplication with input features
3. Apply linear transformation using learned weights
4. Optionally add bias term with broadcasting

#### SimpleGCN Model

The `simpleGCN` class demonstrates a two-layer graph convolutional network:

**Layer Composition**
- First Layer: Graph convolution transforming input features to hidden dimension
 - Applies non-linear ReLU activation
- Second Layer: Additional graph convolution mapping hidden features to output dimension
- Activation: ReLU non-linearity between graph convolution layers


### 4. Train and evaluate simple GCN model on cora dataset




In [2]:
import os
import sys
import numpy as np
sys.path.append('./python')
sys.path.append('./apps')
import needle as ndl
import simple_ml as ml

# process cora dataset
device = ndl.cpu()
content_path = './data/cora/cora.content'
cites_path = './data/cora/cora.cites'
cora_dataset = ndl.data.CoraDataset(content_path, cites_path)
X, y, adjacency_matrix = cora_dataset.get_data()

# initialize DataLoader
X_train, y_train, adj_train, X_test, y_test, adj_test = ml.split_data(X, y, adjacency_matrix)
train_dataloader = ndl.data.DataLoader(X_train, y_train, adj_train, batch_size=64, shuffle=True, device=device)
test_dataloader = ndl.data.DataLoader(X_test, y_test, adj_test, batch_size=64, shuffle=False, device=device)

# simple gnn model
in_features = X.shape[1]
hidden_features = 16
out_features = len(np.unique(y)) # 7
gcn_model = ndl.nn.simpleGCN(in_features, hidden_features, out_features)

# train
trained_gcn = ml.train_gcn(gcn_model, train_dataloader, num_epochs=50, learning_rate=0.005)
# evaluate
accuracy = ml.evaluate_gcn(trained_gcn, test_dataloader)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Using needle backend
Epoch 1/50, Loss: [1.3559676], Train Accuracy: 15.05%
Epoch 2/50, Loss: [1.3445566], Train Accuracy: 15.74%
Epoch 3/50, Loss: [1.3343217], Train Accuracy: 15.51%
Epoch 4/50, Loss: [1.3259437], Train Accuracy: 15.37%
Epoch 5/50, Loss: [1.3173752], Train Accuracy: 15.05%
Epoch 6/50, Loss: [1.3091987], Train Accuracy: 15.10%
Epoch 7/50, Loss: [1.2974317], Train Accuracy: 15.37%
Epoch 8/50, Loss: [1.289646], Train Accuracy: 14.87%
Epoch 9/50, Loss: [1.2811494], Train Accuracy: 14.87%
Epoch 10/50, Loss: [1.2753476], Train Accuracy: 15.56%
Epoch 11/50, Loss: [1.2668805], Train Accuracy: 15.19%
Epoch 12/50, Loss: [1.2627152], Train Accuracy: 14.64%
Epoch 13/50, Loss: [1.25548], Train Accuracy: 15.28%
Epoch 14/50, Loss: [1.2474684], Train Accuracy: 15.19%
Epoch 15/50, Loss: [1.2439557], Train Accuracy: 15.56%
Epoch 16/50, Loss: [1.2382095], Train Accuracy: 15.14%
Epoch 17/50, Loss: [1.2335088], Train Accuracy: 15.19%
Epoch 18/50, Loss: [1.2286911], Train Accuracy: 15.37%
E